# Library Import

In [11]:
import numpy as np
import pandas as pd
from random import random
from random import randint
import math
from random import seed
from random import shuffle

In [35]:
class NeuralNetwork:
    def __init__(self, inputs, hiddens, outputs):
        print(inputs)
        self.n_input = inputs
        self.n_hidden = hiddens
        self.n_output = outputs
        self.network = []
        hidden_layer = [{'weights':[random() for i in range(self.n_input)]} for i in range(self.n_hidden)] #random from num of inputlayer and hiddenlayer (input * hidden)
        self.network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(self.n_hidden)]} for i in range(self.n_output)]
        self.network.append(output_layer)
        self.inputs = []
        self.data = []
        self.best_network = []
        self.testing = []
        
    def compute_net_input(self, weight, inputs):
        net_input = 0
     
        for i in range(len(weight)):
            net_input += weight[i]*inputs[i]
        return net_input

    def sigmoid(self, net_input):
        return 1.0/(1.0 + math.exp(-net_input))

    def forward_propagate(self, data):
        self.inputs = data
        self.data = data
        for layer in self.network:
            next_inputs = []
            for neuron in layer:
                net_input = self.compute_net_input(neuron['weights'], self.inputs)
                neuron['output'] = self.sigmoid(net_input)
                next_inputs.append(neuron['output'])
            self.inputs = next_inputs

    #BackPropagation
    def transfer_derivative(self, output):
        return output * (1.0 - output)

    def back_propagate(self, expected):
        #backprop is begin in outputLayer
        for i in reversed(range(len(self.network))):
            layer = self.network[i]
            errors = []
            if i != len(self.network) - 1: #Hidden Layer
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in self.network[i + 1]:
                        error += neuron['weights'][j] * neuron['errors']
                    errors.append(error)
            else:
                for j in range(len(layer)):
                    neuron = layer[j]
                    errors.append(expected[j] - neuron['output'])
            for j in range(len(layer)):
                neuron = layer[j]
                neuron['errors'] = errors[j] * self.transfer_derivative(neuron['output'])
            
    def update_weights(self, learn_rate):
        for i in range(len(self.network)):
            inputs = self.data[:-1]
            # print(inputs)
            if i != 0:
                inputs = [neuron['output'] for neuron in self.network[i - 1]]
            for neuron in self.network[i]:
                for j in range(len(inputs)):
                    neuron['weights'][j] += learn_rate * neuron['errors'] * inputs[j]
                neuron['weights'][-1] += learn_rate * neuron['errors']

    def training(self, dataset, learn_rate, num_iteration, num_output, tenflow_iterate):
        number_testing = [35*tenflow_iterate, 21*tenflow_iterate] if tenflow_iterate != 9 else [7+(35*tenflow_iterate), 2+(21*tenflow_iterate)]
        testing = [[x for x in range(number_testing[0])], [x for x in range(number_testing[1])]]
        training = []
        for i in range(num_output):
#             for i in range(3):
#                 testing.append(start_index+i)
#                 testing.append(start_index+50+i)
#                 testing.append(start_index+100+i)

            for j in range(len(dataset[i])):
                if j not in testing[i]:
                    training.append(dataset[i][j])
                else:
                    self.testing.append(dataset[i][j])
        
        for iterate in range(num_iteration):
            sum_error = 0
            

            for row in training:
                self.forward_propagate(row)
                expected = [0 for i in range(num_output)]
                expected[row[-1]] = 1
                # print("this is expect ", expected)

                sum_error += sum([(expected[i] - self.inputs[i])**2 for i in range(len(expected))])
                
                self.back_propagate(expected)
                self.update_weights(learn_rate)
           
            print('iteration=%d   learning_rate=%.4f   error=%.4f' % (iterate, learn_rate, sum_error))

    def predict(self, row):
        
        self.forward_propagate(row)
      
        return self.inputs.index(max(self.inputs))

## Define lambda function linear normalize data

In [24]:
normalized = lambda x, maxv, minv : (x-minv*0.95)/(maxv*1.05-minv*0.95)

## Import Data and set number of inputs and outputs

In [31]:
data = pd.read_csv("../wdbc.csv", index_col=0)

num = 0

max_val, min_val = [0 for i in range(30)], [0 for i in range(30)]

for i in data:
    if(i != 'class'):
        if(num == 30):
            break
        max_val[num] = max(data[i])
        min_val[num] = min(data[i])
        num += 1

ID = data.index.values
data_key = []
for j in ID:
    format_data = []
    for i in data:
        format_data.append(data[i][j])
    data_key.append(format_data)


num_inputs = len(data_key[0])-1 

num_outputs = len(set(data['class']))
print(num_inputs, num_outputs)




30 2


In [34]:
for i in range(len(data_key)):
    class_val = data_key[i][0]
    del data_key[i][0]
    data_key[i].append(1 if class_val == 'M' else 0)

#tenflow 35 and 21 and last iteration is 42 and 23

for i in range(len(data_key)):
    for j in range(len(max_val)):
        
        data_key[i][j] = normalized(data_key[i][j], max_val[j], min_val[j])

print("Number of Input Layer: ", num_inputs)
print("Number of Output Layer: ", num_outputs)


print(data_key[0])
shuffle(data_key)
input_data = [[] for _ in range(num_outputs)]
for i in data_key:
    if i[-1] == 0:
        input_data[0].append(i)
    else:
        input_data[1].append(i)
print(len(input_data[0]), len(input_data[1]))

networks = []
accuracy = []

Number of Input Layer:  30
Number of Output Layer:  2
[-0.30211742735522984, -0.2963777917220918, -0.26646598646863073, -0.054680412199359936, 0.3458016304686794, 0.7942636669532198, 5.288529063764034, 3.2168045761575144, 8.65001208552873, -0.31740895635803124, -0.05132739427726211, -0.07768026714785974, -0.03265962148612923, -0.0065571330030386255, 7.827695150142561, 5.704554277563616, 12.49826490590649, 7.273212575281293, 82.44550812936521, -7.999851901957228, -0.25770228736032064, -0.2867248979111518, -0.2220333667644048, -0.04102285904796219, -0.2665970332499976, 0.16632624122106818, 0.7329338036188305, -0.3614240783856547, -0.8470470099475637, -0.31577358125939836, 0]
569 0


In [36]:
for i in range(10):
    networks.append(NeuralNetwork(num_inputs, 10, num_outputs))
    networks[i].training(input_data, 0.1, 500, num_outputs, i)
    
    num = 0
    for row in networks[i].testing:
        prediction = networks[i].predict(row)
        if row[-1] == prediction:
            num += 1
        print("Expect=%d  Output=%d" % (row[-1], prediction))
    accuracy.append(num/15*100)


for i in range(10):
    print("Model ", i)
    print(networks[i].network, end='\n\n')
    print("Accuracy : ", accuracy[i])

print("Mean Accuracy: " ,sum(accuracy)/10)

30
iteration=0   learning_rate=0.1000   error=124.8626
iteration=1   learning_rate=0.1000   error=0.5293
iteration=2   learning_rate=0.1000   error=0.3068
iteration=3   learning_rate=0.1000   error=0.2223
iteration=4   learning_rate=0.1000   error=0.1761
iteration=5   learning_rate=0.1000   error=0.1464
iteration=6   learning_rate=0.1000   error=0.1256
iteration=7   learning_rate=0.1000   error=0.1101
iteration=8   learning_rate=0.1000   error=0.0980
iteration=9   learning_rate=0.1000   error=0.0884
iteration=10   learning_rate=0.1000   error=0.0805
iteration=11   learning_rate=0.1000   error=0.0740
iteration=12   learning_rate=0.1000   error=0.0684
iteration=13   learning_rate=0.1000   error=0.0636
iteration=14   learning_rate=0.1000   error=0.0595
iteration=15   learning_rate=0.1000   error=0.0558
iteration=16   learning_rate=0.1000   error=0.0526
iteration=17   learning_rate=0.1000   error=0.0498
iteration=18   learning_rate=0.1000   error=0.0472
iteration=19   learning_rate=0.1000 

iteration=161   learning_rate=0.1000   error=0.0056
iteration=162   learning_rate=0.1000   error=0.0056
iteration=163   learning_rate=0.1000   error=0.0056
iteration=164   learning_rate=0.1000   error=0.0055
iteration=165   learning_rate=0.1000   error=0.0055
iteration=166   learning_rate=0.1000   error=0.0055
iteration=167   learning_rate=0.1000   error=0.0054
iteration=168   learning_rate=0.1000   error=0.0054
iteration=169   learning_rate=0.1000   error=0.0054
iteration=170   learning_rate=0.1000   error=0.0053
iteration=171   learning_rate=0.1000   error=0.0053
iteration=172   learning_rate=0.1000   error=0.0053
iteration=173   learning_rate=0.1000   error=0.0052
iteration=174   learning_rate=0.1000   error=0.0052
iteration=175   learning_rate=0.1000   error=0.0052
iteration=176   learning_rate=0.1000   error=0.0052
iteration=177   learning_rate=0.1000   error=0.0051
iteration=178   learning_rate=0.1000   error=0.0051
iteration=179   learning_rate=0.1000   error=0.0051
iteration=18

KeyboardInterrupt: 